[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/llamaindex/cookbook/blob/main/integrations/Klavis/Use_Klavis_with_LlamaIndex.ipynb)


# LlamaIndex + Klavis AI Integration

# <img src="../../static/.png" width="500">

In this tutorial, we'll explore how to build powerful AI agents that integrate LlamaIndex's advanced RAG capabilities with Klavis MCP (Model Context Protocol) Servers. This integration combines:

- **LlamaIndex**: Advanced RAG framework with powerful agent capabilities and function calling
- **Klavis AI**: MCP servers for connecting to external tools and services

This combination enables you to create sophisticated agents that can:
- Perform advanced document retrieval and analysis
- Execute complex workflows with external tools
- Maintain conversation context and memory
- Handle multi-step reasoning tasks


## Prerequisites

Before we begin, you'll need:

- **OpenAI API key** - Get yours at [openai.com](https://openai.com/)
- **Klavis API key** - Get yours at [klavis.ai](https://klavis.ai/)

Make sure to keep these API keys secure and never commit them to version control!


In [14]:
# Install the required packages
%pip install -qU llama-index llama-index-tools-mcp requests

ERROR: Ignored the following versions that require a different python version: 0.1.0 Requires-Python <4.0,>=3.10; 0.1.1 Requires-Python <4.0,>=3.10; 0.1.2 Requires-Python <4.0,>=3.10; 0.1.3 Requires-Python <4.0,>=3.10; 0.2.0 Requires-Python <4.0,>=3.10; 0.2.1 Requires-Python <4.0,>=3.10; 0.2.2 Requires-Python <4.0,>=3.10; 0.2.3 Requires-Python <4.0,>=3.10; 0.2.4 Requires-Python <4.0,>=3.10; 0.2.5 Requires-Python <4.0,>=3.10
ERROR: Could not find a version that satisfies the requirement llama-index-tools-mcp (from versions: none)
ERROR: No matching distribution found for llama-index-tools-mcp
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

# Set environment variables
os.environ["OPENAI_API_KEY"] = "your-openai-api-key-here"  # Replace with your actual OpenAI API key
os.environ["KLAVIS_API_KEY"] = "your-klavis-api-key-here"  # Replace with your actual Klavis API key


In [13]:
import requests
import urllib.parse
from typing import Dict, Any, Optional, List
import webbrowser

class KlavisAPI:
    """API Client for Klavis API."""
    
    def __init__(self, api_key: str, base_url: str = "https://api.klavis.ai"):
        self.api_key = api_key
        self.base_url = base_url
        self.headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }
    
    def _make_request(self, method: str, endpoint: str, **kwargs) -> Dict[str, Any]:
        """Make HTTP request with error handling."""
        url = f"{self.base_url}{endpoint}"
        response = requests.request(method, url, headers=self.headers, **kwargs)
        response.raise_for_status()
        return response.json()
    
    def create_mcp_instance(self, server_name: str, user_id: str, platform_name: str) -> Dict[str, str]:
        """Create MCP server instance."""
        data = {
            "serverName": server_name,
            "userId": user_id,
            "platformName": platform_name,
            "connectionType": "StreamableHttp"
        }
        result = self._make_request("POST", "/mcp-server/instance/create", json=data)
        print(f"✅ Created {server_name} MCP instance")
        return {
            'serverUrl': result['serverUrl'],
            'instanceId': result['instanceId']
        }
    
    # Advanced use case if the MCP server requires OAuth authorization
    def redirect_to_oauth(self, instance_id: str, server_name: str) -> None:
        """Open OAuth authorization URL in browser."""
        oauth_url = f"{self.base_url}/oauth/{server_name.lower()}/authorize?instance_id={instance_id}"
        print(f"🔐 Opening OAuth authorization for {server_name}")
        print(f"If you are not redirected automatically, please open this URL: {oauth_url}")
        webbrowser.open(oauth_url)


ModuleNotFoundError: No module named 'requests'

## Case 1 - YouTube AI Agent

create AI agent to summarize youtube video with Llamaindex agent framework and Klavis MCP Server

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.tools.mcp import BasicMCPClient

# 1. Initialize LLM and Klavis API client
llm = OpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))
klavis_api_client = KlavisAPI(api_key=os.getenv("KLAVIS_API_KEY"))

# 2. using Klavis API - Create a YouTube MCP server and get the server URL
youtube_mcp_instance = klavis_api_client.create_mcp_instance(
    server_name="YouTube",
    user_id="1234",
    platform_name="Klavis",
)

youtube_mcp_server_url = youtube_mcp_instance["serverUrl"]

# 3. using LlamaIndex - Create a MCP client
youtube_mcp_client = BasicMCPClient(youtube_mcp_server_url)


In [ ]:
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.tools.mcp import (
    get_tools_from_mcp_url,
    aget_tools_from_mcp_url,
)
youtube_tools = await aget_tools_from_mcp_url(
    youtube_mcp_server_url,
    client=youtube_mcp_client,
)

# 4. using LlamaIndex - Create a YouTube agent
youtube_agent = FunctionAgent(
    name="youtube_agent",
    description="Agent using MCP-based tools",
    tools=youtube_tools,
    llm=llm,
    system_prompt="You are an AI assistant that uses MCP tools."
)


YOUTUBE_VIDEO_URL = "https://www.youtube.com/watch?v=MmiveeGxfX0&t=528s" # pick a video you like!
response = await youtube_agent.run(f"suumarize this video {YOUTUBE_VIDEO_URL}")
print(response)

Congrats! You've succesfully summarize the youtube video you like, save a lot fo times! 

## Case 2 - Workflow: YouTube AI Agent + Email AI Agent

Build Llamaindex AgentWorkflow to summarize youtube video and send it to email

In [ ]:
from llama_index.llms.openai import OpenAI

# 1. Initialize LLM and Klavis API client
llm = OpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))
klavis_api_client = KlavisAPI(api_key=os.getenv("KLAVIS_API_KEY"))

# 2. using Klavis API - Create a YouTube MCP server and get the server URL
youtube_mcp_instance = klavis_api_client.create_mcp_instance(
    server_name="YouTube",
    user_id="1234",
    platform_name="Klavis",
)

# 3. using Klavis API - Create a Gmail MCP server and get the server URL
gmail_mcp_instance = klavis_api_client.create_mcp_instance(
    server_name="Gmail",
    user_id="1234",
    platform_name="Klavis",
)

# 4. Open Gmail OAuth authorization URL
klavis_api_client.redirect_to_oauth(gmail_mcp_instance["instanceId"], "Gmail")

In [ ]:
from llama_index.tools.mcp import BasicMCPClient

youtube_mcp_server_url = youtube_mcp_instance["serverUrl"]
gmail_mcp_server_url = gmail_mcp_instance["serverUrl"]

youtube_mcp_client = BasicMCPClient(youtube_mcp_server_url)
gmail_mcp_client = BasicMCPClient(gmail_mcp_server_url)

youtube_tools = await aget_tools_from_mcp_url(
    youtube_mcp_server_url,
    client=youtube_mcp_client,
)

gmail_tools = await aget_tools_from_mcp_url(
    gmail_mcp_server_url,
    client=gmail_mcp_client,
)


In [ ]:
from llama_index.core.agent.workflow import FunctionAgent

youtube_agent = FunctionAgent(
    name="youtube_agent",
    description="Agent using MCP-based tools",
    tools=youtube_tools,
    llm=llm,
    system_prompt="You are an AI assistant that uses MCP tools.",
    can_handoff_to=["gmail_agent"], # handoff to gmail agent
)

gmail_agent = FunctionAgent(
    name="gmail_agent",
    description="Agent using MCP-based tools",
    tools=gmail_tools,
    llm=llm,
    system_prompt="You are an AI assistant that uses MCP tools."
)

In [ ]:
from llama_index.core.agent.workflow import AgentWorkflow

workflow = AgentWorkflow(
        agents=[youtube_agent, gmail_agent],
        root_agent="youtube_agent",
        # initial_state={"notes": "", "report": ""},
    )

In [ ]:
YOUTUBE_VIDEO_URL = "https://www.youtube.com/watch?v=MmiveeGxfX0&t=528s"

EMAIL_SUBJECT = "Hello, World!"
EMAIL_BODY = "This is a test email."
EMAIL_RECIPIENT = "zihaolin@klavis.ai" # replace with your email

resp = await workflow.run(user_msg=f"suumarize this video {YOUTUBE_VIDEO_URL} and then send it to {EMAIL_RECIPIENT}, subject is {EMAIL_SUBJECT}, body is {EMAIL_BODY}")
print("\n✅ Report:\n", resp.response.content)